### Rerank

WPS process for the [rerank.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/rerank.R) from [ClimDown](https://github.com/pacificclimate/ClimDown)

**Quantile Reranking** fixes bias introduced by the [Climate Analogues (CA)](https://github.com/pacificclimate/ClimDown/blob/master/R/CA.R) step by re-applying a simple quantile mapping bias correction at each grid box

In [1]:
from birdy import WPSClient
from netCDF4 import Dataset
from pkg_resources import resource_filename
from wps_tools.testing import get_target_url
from tempfile import NamedTemporaryFile
import requests
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")
url = "http://localhost:5000/wps"

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [3]:
chickadee = WPSClient(url)

### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.rerank?

Signature:
chickadee.rerank(
    obs_file,
    varname=None,
    qdm_file=None,
    analogues_object=None,
    num_cores='4',
    loglevel='INFO',
    units_bool=True,
    n_pr_bool=True,
    tasmax_units='celsius',
    tasmin_units='celsius',
    pr_units='kg m-2 d-1',
    max_gb=1.0,
    start_date=datetime.date(1971, 1, 1),
    end_date=datetime.date(2005, 12, 31),
    out_file=None,
    analogues_name='analogues',
)
Docstring:
Quantile Reranking fixes bias introduced by the Climate Analogues step

Parameters
----------
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
varname : string
    Name of the NetCDF variable to downscale (e.g. 'tasmax')
out_file : string
    Filename to create with the climate imprint outputs
num_cores : {'1', '2', '3', '4'}positiveInteger
    The number of cores to use for parallel execution
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'

### Run the rerank process

In [5]:
with NamedTemporaryFile(suffix=".nc", prefix="output_", dir="/tmp", delete=True) as out_file:
    output = chickadee.rerank(
        qdm_file=resource_filename("tests", "/data/QDM_expected_output.nc"), 
        obs_file=resource_filename("tests", "/data/tiny_obs.nc"), 
        varname="tasmax",
        out_file=out_file.name,
        num_cores=2,
        analogues_object=resource_filename("tests", "data/analogues.rda")
    )

output_data = output.get()[0]

Access the output with nc_to_dataset() or auto_construct_outputs() from wps_tools.output_handling

In [6]:
# NBVAL_IGNORE_OUTPUT
from wps_tools.output_handling import nc_to_dataset, auto_construct_outputs

output_dataset = nc_to_dataset(output.get()[0])
output_dataset

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    dimensions(sizes): lon(26), lat(26), time(3651)
    variables(dimensions): float64 lon(lon), float64 lat(lat), float64 time(time), float32 tasmax(time,lat,lon)
    groups: 

In [7]:
# NBVAL_IGNORE_OUTPUT
auto_construct_outputs(output.get())

[<class 'netCDF4._netCDF4.Dataset'>
 root group (NETCDF3_CLASSIC data model, file format NETCDF3):
     dimensions(sizes): lon(26), lat(26), time(3651)
     variables(dimensions): float64 lon(lon), float64 lat(lat), float64 time(time), float32 tasmax(time,lat,lon)
     groups: ]

### Test for expected output

In [8]:
expected_data = Dataset(resource_filename("tests","/data/bccaq_expected_output.nc"))
for key, value in expected_data.dimensions.items():
    assert str(output_dataset.dimensions[key]) == str(value)